<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/main/PhotoMedtaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐘 Welcome to Step 4 of Elephant Mining

Congratulations on reaching **Step 4**! By now, you’ve successfully **minted your County Data Group**. In this notebook, you'll use your **seed data** and **property images** to mint your **Photo** and **Photo Data Group**.





## 📥 Step 1: Upload the `.env` File

This notebook requires a `.env` file containing your API keys and credentials. Create a file with the following environment variables:

| Variable Name | Purpose |
|---|---|
| `OPENAI_API_KEY` | Access to OpenAI API |
| `AWS_ACCESS_KEY_ID` | AWS access key |
| `AWS_SECRET_ACCESS_KEY` | AWS secret access key |
| `AWS_DEFAULT_REGION` | AWS REGION |
| `S3_BUCKET_NAME` | Your S3 bucket name |
| `IMAGE_FOLDER_NAME` | Image directory |
| `IMAGES_DIR` | Directory path for images |
| `ELEPHANT_PRIVATE_KEY` | Elephant wallet private key |
| `PINATA_JWT` | Pinata authentication token |

### To upload:
1. Click the **folder icon** 📂 in the left sidebar
2. Click the **"Upload"** button
3. Select your `.env` file

### Example `.env` file:
```env
OPENAI_API_KEY=sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXX
AWS_ACCESS_KEY_ID=XXXXXX
AWS_SECRET_ACCESS_KEY=XXXXXX
S3_BUCKET_NAME=your-s3-bucket-name-here
IMAGES_DIR=images
IMAGE_FOLDER_NAME=images
ELEPHANT_PRIVATE_KEY=xxxxx
PINATA_JWT=xxxxx
```

> ⚠️ **Security Note:** Never commit your `.env` file to version control or share it publicly.


## Step 2: Upload `upload_results.csv`

Upload the `upload_results.csv` file to the `/content/` directory.

> 📌 **Important**: This file was generated by running **Step 2** of the [Seed Data Notebook](https://colab.research.google.com/drive/14tSNSP8Pe-mY4VwX9JhXgfyOvzmN3kC0?usp=sharing#scrollTo=OFKp4E49651Z)

The file should now be downloaded and ready to upload to `/content/upload_results.csv`

## Step 3: Upload `submit.zip`

Upload the `submit.zip` file to the `/content/` directory.

> 📌 **Important**: This file was generated by running **Step 3** of the [County Data Notebook](https://colab.research.google.com/drive/1ZI_eScKFh2kDIZgwXljhOgBIgrenDhRi#scrollTo=HA0ppLFpUm1j)

The file should now be downloaded and ready to upload to `/content/submit.zip`

## Step 4: Verify Data Exists

Once both files are uploaded to `/content/`, you can proceed with the main workflow that depends on these generated datasets.

**Expected file locations:**
- `/content/upload-results.csv`
- `/content/submit.zip`







In [ ]:
!ls -la /content/upload-results.csv
!ls -la /content/submit.zip

##Step 5: Install Package & Setup Folders (Total Runtime ~ 2 minutes)
This step:

Installs the photo-meta-data-ai package from GitHub
Creates all necessary folders for the project
Saves installation details to a log file for troubleshooting

Takes 1-2 minutes to complete. Once finished, you'll have the AI package installed and folder structure ready for processing photos.



In [ ]:
# 1. Install the package
!pip install --force-reinstall --no-cache-dir git+https://github.com/elephant-xyz/photo-meta-data-ai.git > /content/install_log.txt 2>&1

# 2. Set up folders
!colab-folder-setup


##Step 6: Upload Images to Property Subfolders
Upload your property images into the pre-created subfolders:

Each property already has a subfolder named with its Parcel ID under the images folder
Simply drag and drop your images into the correct property subfolder
All images for a specific property should go in that property's designated folder

The AI will process each property's images and generate metadata organized by Parcel ID.






## Step 8 Uploading Photos

In [ ]:
%%bash

# Elephant Fact Sheet Template Installer (Silent)
set -e  # Exit on any error

INSTALL_DIR="${HOME}/.elephant-fact-sheet"
BIN_DIR="${HOME}/.local/bin"
STATIC_TARGET_DIR="$BIN_DIR/templates/assets/static"
TEST="/content/templates/assets/static"

# Check Node.js version
if ! command -v node &> /dev/null; then
    exit 1
fi

NODE_VERSION=$(node -v | cut -d'v' -f2 | cut -d'.' -f1)
if [ "$NODE_VERSION" -lt 18 ]; then
    exit 1
fi

# Check npm
if ! command -v npm &> /dev/null; then
    exit 1
fi

# Clean up any existing installation
rm -rf "$INSTALL_DIR"

# Clone repo silently
git clone --quiet https://github.com/elephant-xyz/fact-sheet-template.git "$INSTALL_DIR"

cd "$INSTALL_DIR"

# Silent npm install and build
npm install --silent --no-audit --no-fund > /dev/null 2>&1
npm run build --silent > /dev/null 2>&1

# Setup directories and copy assets
mkdir -p "$BIN_DIR" "$STATIC_TARGET_DIR" "$TEST"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$STATIC_TARGET_DIR"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$TEST"

# Link and make executable
ln -sf "$INSTALL_DIR/bin/fact-sheet.js" "$BIN_DIR/fact-sheet"
chmod +x "$INSTALL_DIR/bin/fact-sheet.js"

In [ ]:
%%bash

# Step 1: Process photo data
process-photo-data

# Step 2: Unzip the submission archive
unzip submit.zip >> logs/zip.log

# Step 3: Copy all contents from submit/* to photo_data_group/*
for dir in submit/*/; do
  dir_name=$(basename "$dir")
  mkdir -p "photo_data_group/$dir_name"
  cp -r "$dir"* "photo_data_group/$dir_name/"
done


#Step 4: Validate and upload the data
npx -y @elephant-xyz/cli@latest validate-and-upload photo_data_group --output-csv photos.csv



## Step 9: Submitting Your Data to the Blockchain

### Submitting Your Data

Once complete, your data is permanently recorded on the blockchain. You'll receive vMahout tokens as rewards after consensus is reached (when 3 different oracles submit matching data hashes).

In [ ]:
!npx -y @elephant-xyz/cli submit-to-contract photos.csv

## Step 10: Setup and Run AWS Rekognition (Total Runtime ~ 3 minutes)

The system automatically sets up and runs Amazon Rekognition to analyze your property images:

- Connects to AWS Rekognition service for AI-powered image analysis
- Processes all images in your property folders automatically
- Extracts detailed information like room types, architectural features, and property characteristics

No action needed from you - the system handles everything automatically and will notify you when processing is complete.


In [ ]:
!bucket-manager
!unzip-county-data
!upload-to-s3
!photo-categorizer

## Step 11: Running AI to Extract Data from Images (Total Runtime ~ 7 minutes)

The AI system now analyzes your property images to extract valuable metadata:

1. **Image Analysis**: AI examines each photo to identify rooms, features, and property details
2. **Data Extraction**: System pulls out structured information like room types, square footage estimates, architectural elements, and condition assessments

The process runs automatically across all your uploaded property images, generating comprehensive metadata reports for each parcel.



In [ ]:
!pip install --force-reinstall --no-cache-dir git+https://github.com/elephant-xyz/photo-meta-data-ai.git > /content/install_log.txt 2>&1

!ai-analyzer --local-folders --parallel-categories --all-properties
!property-summarizer --all-properties

## Step 12: Data Validation and Submission

The system validates extracted data and prepares it for final submission:

1. **Data Validation**: Reviews and verifies all extracted metadata for accuracy
2. **Submission Preparation**: Validated data is formatted and organized for CLI submission
3. **CLI Submission**: System automatically submits the processed data through the command line interface
4. **Fact Sheet Generation**: Creates comprehensive property fact sheets with all extracted information, images, and metadata

Final deliverables include validated property reports and detailed fact sheets ready for use.


In [ ]:
%%bash

# Elephant Fact Sheet Template Installer (Silent)
set -e  # Exit on any error

INSTALL_DIR="${HOME}/.elephant-fact-sheet"
BIN_DIR="${HOME}/.local/bin"
STATIC_TARGET_DIR="$BIN_DIR/templates/assets/static"
TEST="/content/templates/assets/static"

# Check Node.js version
if ! command -v node &> /dev/null; then
    exit 1
fi

NODE_VERSION=$(node -v | cut -d'v' -f2 | cut -d'.' -f1)
if [ "$NODE_VERSION" -lt 18 ]; then
    exit 1
fi

# Check npm
if ! command -v npm &> /dev/null; then
    exit 1
fi

# Clean up any existing installation
rm -rf "$INSTALL_DIR"

# Clone repo silently
git clone --quiet https://github.com/elephant-xyz/fact-sheet-template.git "$INSTALL_DIR"

cd "$INSTALL_DIR"

# Silent npm install and build
npm install --silent --no-audit --no-fund > /dev/null 2>&1
npm run build --silent > /dev/null 2>&1

# Setup directories and copy assets
mkdir -p "$BIN_DIR" "$STATIC_TARGET_DIR" "$TEST"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$STATIC_TARGET_DIR"
cp -r "$INSTALL_DIR/templates/assets/static/"* "$TEST"

# Link and make executable
ln -sf "$INSTALL_DIR/bin/fact-sheet.js" "$BIN_DIR/fact-sheet"
chmod +x "$INSTALL_DIR/bin/fact-sheet.js"



In [ ]:
!fix-schema-validation
!copy-all-data-for-submission
!copy-all-files-from-zip
!npx @elephant-xyz/cli@latest validate-and-upload submit-photo --output-csv submit-results.csv

## Step 13: Submitting Your Data to the Blockchain

### Submitting Your Data

Once complete, your data is permanently recorded on the blockchain. You'll receive vMahout tokens as rewards after consensus is reached (when 3 different oracles submit matching data hashes).

In [ ]:
!npx -y @elephant-xyz/cli submit-to-contract submit-results.csv

## Step 14: Package and Download Results

This step creates downloadable files with all your processed data. The system will:

1. **Create Download Package**: Automatically zip the submit-photos folder containing all fact sheets and processed images

2. **Download Results**: Two files will be made available for download

**Files to Download:**
- `submit-results.csv` - Structured data with all extracted property metadata
- `submit-photo.zip` - Complete package containing fact sheets and processed images

Your processed property data is now saved locally for use.

In [ ]:
!zip -r submit-photo.zip submit-photo/ > /dev/null 2>&1



## Step 15: Cleanup

Final and optional step to save your results and clean up the workspace:

3. **Cleanup Workspace**: After downloading, the system removes all temporary files and folders including:
  - `images` folder (uploaded property photos)
  - `output` folder (processing files)
  - `county-data` folder (temporary data)
  - `submit-photos` folder (final results)
  - `logs` folder (processing logs)

**Important**: Make sure to download your results before the cleanup step, as all files will be permanently deleted from the workspace.

In [ ]:
!rm -rf images/ output/ county-data/ submit/ submit-photo/ logs/ photo_data_group/ > /dev/null 2>&1
!find . -maxdepth 1 -type f \( -name "*.csv" -o -name "*.zip" -o -name "*.jpg" -o -name "*.txt" -o -name "*.log" -o -name ".env" \) -exec rm -f {} \;

!rm -rf /root/.local/bin/fact-sheet
!rm -rf fact-sheet-template/
!rm -rf /root/.elephant-fact-sheet